# Influencer Recommendation

This notebook was used to combine financial information and tweets with sentiment scoring.

These two datasets were joined by date, and then used to evaluate which Twitter accounts had a best correlation with stock movements; i.e. which accounts had tweets with positive sentiment on days that the stock price went up, and negative sentiment on days that the stock price went down.

In [1]:
import pandas as pd

Download the datasets to the cluster where the user can find them

In [2]:
!aws s3 cp s3://team75data/AAPL_sentiment.csv /tmp/AAPL_sentiment.csv
!aws s3 cp s3://team75data/finance_data/AAPL.csv /tmp/finance_data/AAPL.csv
!aws s3 cp s3://team75data/AMZN_sentiment.csv /tmp/AMZN_sentiment.csv
!aws s3 cp s3://team75data/finance_data/AMZN.csv /tmp/finance_data/AMZN.csv
!aws s3 cp s3://team75data/GOOGL_sentiment.csv /tmp/GOOGL_sentiment.csv
!aws s3 cp s3://team75data/finance_data/GOOGL.csv /tmp/finance_data/GOOGL.csv
!aws s3 cp s3://team75data/MSFT_sentiment.csv /tmp/MSFT_sentiment.csv
!aws s3 cp s3://team75data/finance_data/MSFT.csv /tmp/finance_data/MSFT.csv
!aws s3 cp s3://team75data/TSLA_sentiment.csv /tmp/TSLA_sentiment.csv
!aws s3 cp s3://team75data/finance_data/TSLA.csv /tmp/finance_data/TSLA.csv    

download: s3://team75data/AAPL_sentiment.csv to tmp/AAPL_sentiment.csv
download: s3://team75data/finance_data/AAPL.csv to tmp/finance_data/AAPL.csv
download: s3://team75data/AMZN_sentiment.csv to tmp/AMZN_sentiment.csv
download: s3://team75data/finance_data/AMZN.csv to tmp/finance_data/AMZN.csv
download: s3://team75data/GOOGL_sentiment.csv to tmp/GOOGL_sentiment.csv
download: s3://team75data/finance_data/GOOGL.csv to tmp/finance_data/GOOGL.csv
download: s3://team75data/MSFT_sentiment.csv to tmp/MSFT_sentiment.csv
download: s3://team75data/finance_data/MSFT.csv to tmp/finance_data/MSFT.csv
download: s3://team75data/TSLA_sentiment.csv to tmp/TSLA_sentiment.csv
download: s3://team75data/finance_data/TSLA.csv to tmp/finance_data/TSLA.csv


Rotate throught the different companies

In [3]:
#company = "AAPL"
#company = "AMZN"
company = "GOOGL"
#company = "MSFT"
#company = "TSLA"

Load sentiment and financial data

In [4]:
sentiment_df = pd.read_csv("/tmp/"+company+"_sentiment.csv")
finance_df = pd.read_csv("/tmp/finance_data/"+company+".csv")

In [5]:
sentiment_df.head()

,Unnamed: 0,ticker_symbol,tweet_id,writer,text,comment_num,retweet_num,like_num,company_name,sector,year,month,day,hour,sentiment
0,129,GOOGL,1154702739332579329,SamUnsted,"Google-owner Alphabet up 8.8% pre-mkt, implyin...",0,0,1,Google Inc,Communication Services,2019,7,26,7,1
1,156,GOOGL,692905805314703360,TalkMarkets,"Amazon Earnings Miss, Blown Up A/H $AMZN $SPY ...",0,0,0,Google Inc,Communication Services,2016,1,29,0,1
2,191,GOOGL,880487914005823490,paszportWS,#Reklama #Google $GOOGL,0,0,0,Google Inc,Communication Services,2017,6,29,15,1
3,209,GOOGL,1025734366578855938,OptionAlarm,"With past performance like this, how can you n...",0,0,1,Google Inc,Communication Services,2018,8,4,10,1
4,219,GOOGL,1115590243145535489,pancholig1,Top Video Game Stocks for 2019 and Beyond @the...,0,0,0,Google Inc,Communication Services,2019,4,9,9,1


Select some columns

In [6]:
sentiment_df_trim = sentiment_df[["writer", "year", "month", "day", "sentiment"]]

In [7]:
sentiment_df_trim.describe()

,year,month,day,sentiment
count,320503.000000,320503.000000,320503.000000,320503.000000
mean,2016.923916,6.434564,16.180045,0.856965
std,1.443779,3.318309,8.968502,0.350109
min,2014.000000,1.000000,1.000000,0.000000
25%,2016.000000,4.000000,8.000000,1.000000
50%,2017.000000,7.000000,17.000000,1.000000
75%,2018.000000,9.000000,24.000000,1.000000
max,2021.000000,12.000000,31.000000,1.000000


In [8]:
finance_df_trim = finance_df[["Date", "Open", "Close"]]

In [9]:
finance_df_trim["Delta"] = finance_df_trim.Close - finance_df_trim.Open

/emr/notebook-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
def sign(value):
    # Calculate delta sign
    if value >= 0:
        return 1
    else:
        return 0

finance_df_trim["Delta_sign"] = finance_df_trim.apply(lambda row: sign(row.Delta), axis=1)

/emr/notebook-env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
def make_date(year, month, day):
    # Create date in the same format as the finance dataset
    if len(str(day)) == 1:
        day = "0" + str(day)
    if len(str(month)) == 1:
        month = "0" + str(month)
        
    return str(year) + "-" + str(month) + "-" + str(day)

sentiment_df_trim["Date"] = sentiment_df_trim.apply(lambda row: make_date(row.year, row.month, row.day), axis=1)

sentiment_df_trim = sentiment_df_trim[["writer", "sentiment", "Date"]]

/emr/notebook-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
finance_df_trim = finance_df_trim[["Date", "Delta_sign"]]

In [13]:
finance_df_trim['Date'] = finance_df_trim['Date'].astype(str)
sentiment_df_trim['Date'] = sentiment_df_trim['Date'].astype(str)

In [14]:
merged_sentiment_and_delta = pd.merge(sentiment_df_trim, finance_df_trim, how='left')

In [15]:
writers = merged_sentiment_and_delta[["writer"]]

In [16]:
number_of_tweets = writers.groupby("writer").size().reset_index(name='Tweets')

Keep only writers that write often (over 150 tweets).

In [17]:
authors = list(number_of_tweets[number_of_tweets['Tweets'] > 150].writer)

In [18]:
merged_sentiment_and_delta[merged_sentiment_and_delta.writer.isin(authors)]

,writer,sentiment,Date,Delta_sign
1,TalkMarkets,1,2016-01-29,1.0
3,OptionAlarm,1,2018-08-04,NaN
6,businessinsider,1,2016-09-23,0.0
11,OACtrading,1,2019-01-27,NaN
16,FinInvTr,1,2017-11-30,0.0
...,...,...,...,...
320494,OACtrading,1,2018-11-07,1.0
320496,stockspastor,1,2019-03-18,0.0
320498,EliteOptionSwap,1,2019-05-23,0.0
320500,clayton_dd,1,2015-09-27,NaN


In [19]:
author_correlations = {}
for author in authors:
    tmp = merged_sentiment_and_delta[merged_sentiment_and_delta.writer == author].dropna()
    a = tmp[["sentiment", "Delta_sign"]].corr()["sentiment"]["Delta_sign"]
    author_correlations[author] = a

In [20]:
import operator

best_authors = []
for i in range(10):
    author = max(author_correlations.items(), key=operator.itemgetter(1))
    best_authors.append(author)
    author_correlations.pop(author[0], None)

best_authors

[('Terri1618', 0.21813530471996911),
 ('SunAndStormInv', 0.20443085178626252),
 ('andrewross11', 0.19995229072207096),
 ('HedgeBz', 0.19940936336117157),
 ('zen_n_trade', 0.18140157331255047),
 ('OptionSamurai', 0.18122861779254498),
 ('Mr_MoonStocks', 0.17768008300898774),
 ('Tickeron', 0.17170902802382237),
 ('LlcBillionaire', 0.16246792326664972),
 ('Trading_Analyst', 0.15554640283564644)]

In [21]:
print("company,usr_handle,correlation,tweet_count")
for author in best_authors:
    print(company + "," + author[0] + "," + str(author[1]) + "," + str(float(number_of_tweets[number_of_tweets["writer"] == author[0]]["Tweets"]))) 
    

company,usr_handle,correlation,tweet_count
GOOGL,Terri1618,0.21813530471996911,194.0
GOOGL,SunAndStormInv,0.20443085178626252,181.0
GOOGL,andrewross11,0.19995229072207096,229.0
GOOGL,HedgeBz,0.19940936336117157,224.0
GOOGL,zen_n_trade,0.18140157331255047,157.0
GOOGL,OptionSamurai,0.18122861779254498,180.0
GOOGL,Mr_MoonStocks,0.17768008300898774,151.0
GOOGL,Tickeron,0.17170902802382237,173.0
GOOGL,LlcBillionaire,0.16246792326664972,176.0
GOOGL,Trading_Analyst,0.15554640283564644,500.0
